In [1]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import pandas as pd
import scipy
from sympy.matrices import Matrix, eye
#
from SBMLLint.games.message import Message

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [2]:
simple = load_file_from_curated_data(40)
for r in simple.reactions:
  #if r.category != cn.REACTION_BOUNDARY:
    print(r.makeIdentifier(is_include_kinetics=False))

Reaction1: Br + BrO3 -> HBrO2 + HOBr
Reaction2: Br + HBrO2 -> 2.00 HOBr
Reaction3: BrO3 + HBrO2 -> Ce + 2.00 HBrO2
Reaction4: 2.00 HBrO2 -> BrO3 + HOBr
Reaction5: Ce -> Br


In [3]:
m = Message(simple)
m

{HBrO2};{BrO3};{Ce};{Br};{HOBr}

In [4]:
m.stoichiometry_matrix

,Reaction1,Reaction2,Reaction3,Reaction4,Reaction5
HBrO2,1.0,-1.0,1.0,-2.0,0.0
BrO3,-1.0,0.0,-1.0,1.0,0.0
Ce,0.0,0.0,1.0,0.0,-1.0
Br,-1.0,-1.0,0.0,0.0,1.0
HOBr,1.0,2.0,0.0,1.0,0.0


In [5]:
m.reactions

[Reaction1: Br + BrO3 -> HBrO2 + HOBr; Br * BrO3 * k1 * BZ,
 Reaction2: Br + HBrO2 -> 2.00 HOBr; Br * HBrO2 * k2 * BZ,
 Reaction3: BrO3 + HBrO2 -> Ce + 2.00 HBrO2; BrO3 * HBrO2 * k3 * BZ,
 Reaction4: 2.00 HBrO2 -> BrO3 + HOBr; pow(HBrO2, 2) * k4 * BZ,
 Reaction5: Ce -> Br; Ce * k5 * BZ]

In [6]:
m.molecules

[HBrO2, BrO3, Ce, Br, HOBr]

In [7]:
m.stoichiometry_matrix

,Reaction1,Reaction2,Reaction3,Reaction4,Reaction5
HBrO2,1.0,-1.0,1.0,-2.0,0.0
BrO3,-1.0,0.0,-1.0,1.0,0.0
Ce,0.0,0.0,1.0,0.0,-1.0
Br,-1.0,-1.0,0.0,0.0,1.0
HOBr,1.0,2.0,0.0,1.0,0.0


In [8]:
mat_df = m.stoichiometry_matrix
m.decomposeMatrix(mat_df).T

,HBrO2,BrO3,Ce,Br,HOBr
Reaction1,1.0,-1.0,0.0,-1.0,1.0
Reaction2,0.0,-1.0,0.0,-2.0,3.0
Reaction3,0.0,0.0,1.0,1.0,-1.0
Reaction5,0.0,0.0,0.0,2.0,-1.0
Reaction4,0.0,0.0,0.0,0.0,0.0


In [9]:
m.permuted_matrix.T

,HBrO2,BrO3,Ce,Br,HOBr
Reaction1,1.0,-1.0,0.0,-1.0,1.0
Reaction2,-1.0,0.0,0.0,-1.0,2.0
Reaction3,1.0,-1.0,1.0,0.0,0.0
Reaction5,0.0,0.0,-1.0,1.0,0.0
Reaction4,-2.0,1.0,0.0,0.0,1.0


In [10]:
m.lower_inverse

,Reaction1,Reaction2,Reaction3,Reaction5,Reaction4
0,1.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0
2,-1.0,0.0,1.0,0.0,0.0
3,-1.0,0.0,1.0,1.0,0.0
4,1.0,-1.0,0.0,0.0,1.0


In [11]:
m.convertMatrixToReactions(simple, )

TypeError: convertMatrixToReactions() missing 2 required positional arguments: 'simple' and 'mat_df'

In [32]:
# mat_df = m.stoichiometry_matrix
# mat_t = mat_df.T

# mat_t
# idx_mat_t = mat_t.index
# # # LU decomposition
# mat_lu = scipy.linalg.lu(mat_t)
# # # inverse pivot matrix
# p_inv = scipy.linalg.inv(mat_lu[0])
# pivot_index = [list(k).index(1) for k in p_inv]
# new_idx_mat_t = [idx_mat_t[idx] for idx in pivot_index]
# # # row reduced matrix
# row_reduced = pd.DataFrame(mat_lu[2], index=new_idx_mat_t, columns=mat_t.columns)
# row_reduced